# Setting up FusionSC

## Setting up component lookup (version 2.2 and above)

While FusionSC can be used out of the box (`import fusionsc as fsc`), it doesn't bundle the part and coil geometries (partly due to size, partly to comply with data access policies). In order to run FusionSC with pre-defined geometries, the resolution subsystem needs to be hooked up with lookup tables / storage for these devices.

The recommended way (available from version 2.2 onwards) is to use the user configuration file in "~/.fusionsc.yaml" (an alternative path can be specified by setting the FUSIONSC_CONFIG_PATH environment variable). The "fusionsc-config" command line (alternatively usable via "python -m fusionsc.config") can be used to manage this configuration file.

### Wendelstein 7-X inside IPP network

Use the "ipp-hgw" default profile, which will connect to a compute server and rely on the resolution database stored there.

```
fusionsc-config default ipp-hgw
```

### J-TEXT

Simplified J-TEXT geometries are bundled with FusionSC. No further configuration is required to use them.

### Others

You will have to manually add the neccessary resolution files. Usually this entails adding them with the  config tool

```
fusionsc-config resolve add {file or url}
```

## Setting up component lookup with python methods (version 2.0 and above)

For version 2.1 and below, and if you don't want to rely on the user configuration, you can always use the methods `fusionsc.resolve.importOfflineData` and `fusionsc.resolve.connectWarehouse`. Keep in mind that the changes by these methods are local to the current context.

### Wendelstein 7-X inside IPP network

On version 2.1 and above, you can use the `fusionsc.devices.w7x.connectIppSite` method to configure the computing server and resolution database, similar to the "ipp-hgw" configuration default profile.

In [1]:
from fusionsc.devices import w7x
w7x.connectIppSite()

## (W7-X only) Pre-calculating fields for Biot-Savart calculation

A generally expensive calculation is the Biot-Savart rule to obtain the magnetic field from the coil geometries. For W7-X
, there is a high-level support to precompute the coil fields. These fields can then be saved and loaded later for
usage. When using precomputed coil fields, the offline data files are not required (unless required for other reasons, such
as geometry data or other field information). W7-X has special support to pre-compute all coils so that they can be re-used
later.

The precomputation is performed through the `devices.w7x.CoilPack` class, which can be used as a handle to override the coils
used in the generation of W7-X configurations.

In [1]:
from fusionsc.devices import w7x

# Obtain default coils
coils = w7x.cadCoils()

# Compute
coils = coils.computeFields(w7x.defaultGrid())

# Use in configuration
config1 = w7x.standard(coils = coils)
config2 = w7x.highIota(coils = coils)

The precalculated fields can be saved and loaded like other field or geometry classes.

In [ ]:
# Save
coils.save("coils.fsc")

# Restore
coils = w7x.CoilPack.load("coils.fsc")

## (Re-)Configuring the local backend

While fusionsc is capable of connecting to remote servers for execution and data management, by default it uses a local backend started at import time. The backend is started with a default configuration (all services enabled, CPU execution, auto-detected no. of computation threads). Sometimes it is desirable to override this default behavior. You can use the `backends.reconfigureLocalBackend` function for this purpose.

*Note*: This function creates a **new** backend to be used by all future calls. The old backend will be closed once all references to it coming from previous pending calls are dropped. If the local backend has been exposed (e.g. as a network service or by storing the result of backends.activeBackend) it will not shut down until these external references are dropped as well.

In [2]:
# Get default configuration
config = fsc.service.LocalConfig.newMessage()
print(' --- Default configuration --- ')
print()
print(config)
print()

# Adjust configuration
config.cpuBackend.numThreads.fixed = 2

print(' --- Adjusted configuration --- ')
print()
print(config)
print()

# Apply adjusted configuration
fsc.backends.reconfigureLocalBackend(config)

 --- Default configuration --- 

preferredDeviceType: cpu
enableCompute: true
enableStorage: true
jobScheduler: system
flt:
  eventBuffer:
    minSize: 100
    maxSize: 2500
    targetTotalMb: 500
cpuBackend:
  numThreads: autoDetect

 --- Adjusted configuration --- 

preferredDeviceType: cpu
enableCompute: true
enableStorage: true
jobScheduler: system
flt:
  eventBuffer:
    minSize: 100
    maxSize: 2500
    targetTotalMb: 500
cpuBackend:
  numThreads:
    fixed: 2



## Using verbose logging

Sometimes it is desirable to enable the internal log output to see progress on the calculation. This can be achieved by setting the `FUSIONSC_VERBOSE` environment variable to any other value than `0`. This has to be done **before** importing the package.

In [1]:
import os
os.environ['FUSIONSC_VERBOSE'] = '1'

import fusionsc as fsc

When using the standalone server, verbose logging can be activated through the `--verbose` command-line switch.